In [1]:
import warnings

warnings.simplefilter(action='ignore', category=Warning)

import pandas as pd
import numpy as np

train_df = pd.read_csv('Datasets/train.csv')
test_df = pd.read_csv('Datasets/test.csv')

In [10]:
name_df = train_df['Name']
titles = name_df.apply(lambda x: x.split(',')[1].split('.')[0].strip())
title_list = np.unique(titles)

train_df['titles'] = titles


title_grp = train_df.groupby('titles')

title_age_dict = {}

for title in title_list:
    title_age_dict[title] = title_grp.get_group(title)['Age'].mean()

title_age_dict = {k:round(title_age_dict[k], 1) for k in title_age_dict}


title_age_dict

{'Capt': 70.0,
 'Col': 58.0,
 'Don': 40.0,
 'Dr': 42.0,
 'Jonkheer': 38.0,
 'Lady': 48.0,
 'Major': 48.5,
 'Master': 4.6,
 'Miss': 21.8,
 'Mlle': 24.0,
 'Mme': 24.0,
 'Mr': 32.4,
 'Mrs': 35.9,
 'Ms': 28.0,
 'Rev': 43.2,
 'Sir': 49.0,
 'the Countess': 33.0}

In [12]:
'''
now replacing all NaN values in age column
'''
""" for title in title_list:
    ages = title_grp.get_group(title)['Age']
    if ages.hasnans == True:
        ages = ages.fillna(title_age_dict[title])
        train_df['Age'].loc[ages.index.values] = ages.values """

" for title in title_list:\n    ages = title_grp.get_group(title)['Age']\n    if ages.hasnans == True:\n        ages = ages.fillna(title_age_dict[title])\n        train_df['Age'].loc[ages.index.values] = ages.values "

In [3]:
def df_cleaner(df):
    df['Sex'] = df['Sex'].replace({'male': 1, 'female': 2})

    name_df = df['Name']
    titles = name_df.apply(lambda x: x.split(',')[1].split('.')[0].strip())
    title_list = np.unique(titles)

    df['titles'] = titles

    title_grp = df.groupby('titles')
    for title in title_list:
        ages = title_grp.get_group(title)['Age']
        if ages.hasnans == True:
            ages = ages.fillna(title_age_dict[title])
            df['Age'].loc[ages.index.values] = ages.values

    return df



In [4]:
train_df = df_cleaner(train_df)
test_df = df_cleaner(test_df)

In [6]:
x_train_df = train_df.iloc[:, [2, 4, 5]].values

y_train_df = train_df.iloc[:, [1]].values

x_test_df = test_df.iloc[:, [1, 3, 4]].values

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train_df, y_train_df, test_size = 0.25, random_state = 0)


In [7]:
from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train) 

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

from sklearn.metrics import accuracy_score

print ("Accuracy : ", accuracy_score(y_test, y_pred))

Accuracy :  0.6278026905829597


In [8]:
x_test = sc_x.transform(x_test_df)

y_pred = classifier.predict(x_test)

In [9]:

submission = pd.DataFrame(test_df['PassengerId'])
submission['Survived'] = y_pred

submission.to_csv('submission_random_forest.csv', index=False)